In [1]:
import pandas as pd
import os
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
#TSB'den yayınlanan aylık raporlar TSB-Data klasörüne eklenir.
path = r'TSB-Data'

folders = os.listdir(path)

#Her klasör YYYYMM şeklinde isimlendirilir. Örneğin 2021 Haziran -> 202106 gibi
for item in folders:
    if item.isnumeric():
        pass
    else:
        to_pop = folders.index(item)
        folders.pop(to_pop)
        
#Boş bir dataframe oluşturularak her bir rapordan değerler alınarak bu dataframe'e eklenir.
data = pd.DataFrame()

for item in folders:
    files = os.listdir(path + '/' + item)
    for file in files:
        if file[0] == '3':
            df = pd.read_excel(path + '/' + item + '/' + file, sheet_name = 'Hayat', usecols = 'B, D', skiprows = 5)
            df.rename(columns={'Şirket Adı': 'COMPANY', 'Toplam Üretim (TL)': 'PREMIUM'}, inplace = True)
            df['YEAR'] = int(item[:4])
            df['MONTH'] = int(item[4:])

            df.drop(df.index[df['COMPANY'] == 'İLK 10 ŞİRKET'], inplace = True)
            df.drop(df.index[df['COMPANY'] == 'İLK 20 ŞİRKET'], inplace = True)
            df.drop(df.index[df['COMPANY'] == 'SEKTÖR TOPLAMI'], inplace = True)

            df.replace('AvivaSA Emeklilik ve Hayat AŞ', 'AgeSA Hayat ve Emeklilik AŞ', inplace = True)

            data = data.append(df, ignore_index = True)

In [3]:
data.sort_values(['MONTH', 'PREMIUM'], inplace = True, ascending = [True, False])

data.reset_index(inplace = True, drop = True)

data['PREMIUM'] = np.where(
    data.groupby(['COMPANY', 'YEAR'])['PREMIUM'].diff().fillna(0) == 0, 
    data['PREMIUM'], 
    data.groupby(['COMPANY', 'YEAR'])['PREMIUM'].diff().fillna(0)
)

data.head(25)

,COMPANY,PREMIUM,YEAR,MONTH
0,Türkiye Hayat ve Emeklilik AŞ,"266,659,046.92",2021,1
1,Allianz Yaşam ve Emeklilik AŞ,"119,068,506.88",2021,1
2,AgeSA Hayat ve Emeklilik AŞ,"110,910,681.07",2021,1
3,Anadolu Hayat Emeklilik AŞ,"104,040,979.55",2021,1
4,Garanti Emeklilik ve Hayat AŞ,"102,888,745.11",2021,1
5,Metlife Emeklilik ve Hayat AŞ,"102,523,992.74",2021,1
6,Aegon Emeklilik ve Hayat AŞ,"94,726,624.96",2021,1
7,Cigna Sağlık Hayat ve Emeklilik AŞ,"63,254,018.89",2021,1
8,BNP Paribas Cardif Hayat Sigorta AŞ,"38,233,355.10",2021,1
9,Fiba Emeklilik ve Hayat AŞ,"35,762,194.94",2021,1


In [4]:
data.pivot_table(values = 'PREMIUM', index = 'COMPANY', columns = ['YEAR' , 'MONTH'], aggfunc = 'sum')

YEAR                                          2021                 \
MONTH                                            1              2   
COMPANY                                                             
Aegon Emeklilik ve Hayat AŞ          94,726,624.96  96,960,766.69   
AgeSA Hayat ve Emeklilik AŞ         110,910,681.07 129,062,190.94   
Allianz Hayat ve Emeklilik AŞ        12,822,388.24  16,405,728.21   
Allianz Yaşam ve Emeklilik AŞ       119,068,506.88 127,950,348.05   
Anadolu Hayat Emeklilik AŞ          104,040,979.55 119,983,398.80   
Axa Hayat ve Emeklilik AŞ             1,101,817.21     966,803.79   
BNP Paribas Cardif Emeklilik AŞ      28,131,817.01  40,671,551.04   
BNP Paribas Cardif Hayat Sigorta AŞ  38,233,355.10  37,752,772.75   
Bereket Emeklilik ve Hayat AŞ        12,508,039.00  14,391,575.21   
Bereket Katılım Hayat AŞ                757,282.85     636,074.04   
Bupa Acıbadem Sigorta AŞ              1,305,976.06      50,048.51   
Cigna Sağlık Hayat ve Emeklilik AŞ   63,254,018.89  73,319,601.35   
Demir Sağlık ve Hayat Sigorta AŞ         53,258.06      46,169.74   
Fiba Emeklilik ve Hayat AŞ           35,762,194.94  40,412,310.69   
Garanti Emeklilik ve Hayat AŞ       102,888,745.11 109,909,021.25   
Groupama Hayat AŞ                     1,470,314.24   6,182,897.43   
Katılım Emeklilik ve Hayat AŞ         5,687,354.74   7,357,021.53   
Mapfre Yaşam Sigorta AŞ               7,138,951.43   8,155,892.93   
Metlife Emeklilik ve Hayat AŞ       102,523,992.74  94,324,026.71   
NN Hayat ve Emeklilik AŞ             11,332,643.07  16,702,517.41   
Türkiye Hayat ve Emeklilik AŞ       266,659,046.92 280,548,433.91   

YEAR                                                               \
MONTH                                            3              4   
COMPANY                                                             
Aegon Emeklilik ve Hayat AŞ         116,469,972.82 106,750,885.05   
AgeSA Hayat ve Emeklilik AŞ         148,649,350.05 137,968,008.65   
Allianz Hayat ve Emeklilik AŞ        25,042,083.58  18,709,932.70   
Allianz Yaşam ve Emeklilik AŞ       169,636,126.69 125,924,439.28   
Anadolu Hayat Emeklilik AŞ          169,507,995.70 115,573,124.81   
Axa Hayat ve Emeklilik AŞ             2,038,454.00     887,353.66   
BNP Paribas Cardif Emeklilik AŞ      45,555,405.31  30,397,104.29   
BNP Paribas Cardif Hayat Sigorta AŞ  44,028,197.91  34,680,103.11   
Bereket Emeklilik ve Hayat AŞ        21,137,086.05  13,788,399.74   
Bereket Katılım Hayat AŞ              2,896,679.84   2,573,910.99   
Bupa Acıbadem Sigorta AŞ                 34,457.43      27,350.21   
Cigna Sağlık Hayat ve Emeklilik AŞ   84,011,291.06  63,896,491.60   
Demir Sağlık ve Hayat Sigorta AŞ         50,880.97      49,376.66   
Fiba Emeklilik ve Hayat AŞ           49,223,046.97  37,972,166.48   
Garanti Emeklilik ve Hayat AŞ       111,747,699.14  84,363,945.50   
Groupama Hayat AŞ                     2,469,748.28   1,344,826.49   
Katılım Emeklilik ve Hayat AŞ         9,664,606.48   7,055,928.29   
Mapfre Yaşam Sigorta AŞ               6,445,828.43   5,659,029.96   
Metlife Emeklilik ve Hayat AŞ       145,132,569.12 109,707,124.93   
NN Hayat ve Emeklilik AŞ             14,928,524.55  10,397,872.25   
Türkiye Hayat ve Emeklilik AŞ       360,787,835.96 240,565,906.74   

YEAR                                                               \
MONTH                                            5              6   
COMPANY                                                             
Aegon Emeklilik ve Hayat AŞ         108,196,841.51 136,397,651.31   
AgeSA Hayat ve Emeklilik AŞ         141,257,999.72 165,719,612.17   
Allianz Hayat ve Emeklilik AŞ        13,954,853.77  21,712,928.87   
Allianz Yaşam ve Emeklilik AŞ       125,485,184.67 159,866,805.23   
Anadolu Hayat Emeklilik AŞ           97,812,463.06 187,793,809.96   
Axa Hayat ve Emeklilik AŞ            10,445,413.92   1,803,746.32   
BNP Paribas Cardif Emeklilik AŞ      34,495,231.27  55,166,536.78

In [5]:
data.to_clipboard(index = False)